# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 3-1. Задача TSP: инкрементальные алгоритмы.

В этой задаче Вам предлагается сравнить алгоритмы Nearest Neighbour и Nearest Insertion в задаче Euclidean TSP.

**Даны:**
* Координаты точек плоскости, являющихся вершинами графа.

**Найти:**
* Перестановку вершин, задающих минимальный по длине гамильтонов цикл в графе.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP.
* Реализуйте функции `solve_tsp_nearest_neighbour` и `solve_tsp_nearest_insertion`.
* Запустите функцию `run_all()`, чтобы протестировать свой код и сравнить качество решений, получаемых Nearest Neighbour и Nearest Insertion. Сильно ли они отличаются? Запишите свои качественные выводы в 1-2 предложениях в последней ячейке ipynb-файла.

**Модули**

In [62]:
import numpy as np
from math import sqrt

**Служебные функции**

In [94]:
def read_tsp_instance(filename):
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0,0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates

def euclidean_distance(point1, point2):
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)
    
def calculate_tour_length(instance, permutation):
    n = len(permutation)
    return sum(euclidean_distance(instance[permutation[i]], instance[permutation[(i+1) % n]]) 
               for i in range(len(permutation)))

def weight(i, j, array):
        return euclidean_distance(array[i], array[j])

Решаем задачу методом **Nearest Neighbour**. Начиная с произвольной вершины, на каждой итерации находим ближайшую к текущей и добавляем ее в наш цикл. 

In [95]:
def solve_tsp_nearest_neighbour(instance):
    
    to_visit = instance.copy()
    current_vertex = to_visit.pop()
    tour = [instance.index(current_vertex)]
    
    for iteration_num in range(len(instance) - 1):
        nearest_vertex = min(to_visit, key=lambda v: euclidean_distance(v, current_vertex))
        tour.append(instance.index(nearest_vertex))
        to_visit.remove(nearest_vertex)
        current_vertex = nearest_vertex
        
    return tour

Решаем задачу методом **Nearest Insertion**. На каждом шаге, уже имея гамильтонов цикл на некотором подмножестве вершин графа, будем искать ближайшую к этому циклу вершину. Затем, вставляем ее в цикл, найдя ребро, расщепив которое и добавив два новых ребра, мы прибавим минимальный вес.

In [106]:
def solve_tsp_nearest_insertion(instance):
    
    instance = np.array(instance)
    size = len(instance)
    tour = np.arange(0, 1)
    to_visit = np.arange(1, size)
    
    for tour_length in range(1, size):
        
        in_tour = instance[tour].T
        not_in_tour = instance[to_visit].T
        
        temporary_column = np.tile(in_tour, size - tour_length)
        temporary_column = temporary_column.reshape((2, size - tour_length, tour_length))
        temporary_column = np.array([column.T for column in temporary_column])
        reshape_temporary = np.tile(not_in_tour, tour_length).reshape((2, tour_length, size - tour_length))
        temporary_distances = np.sqrt((temporary_column[0] - reshape_temporary[0]) ** 2 +
                                      (temporary_column[1] - reshape_temporary[1]) ** 2)
        distances = temporary_distances + np.roll(temporary_distances, 1, axis=0)
        neighbour_distances = np.array([weight(tour[i - 1], tour[i], instance) for i in range(tour_length)])
        matrix = distances.T - neighbour_distances
        arg_min = np.argmin(matrix)
        new_tour_edge = arg_min % tour_length
        to_visit_num = arg_min // tour_length
        tour = np.insert(tour, new_tour_edge, to_visit[to_visit_num])
        to_visit = np.delete(to_visit, to_visit_num)
        
    return tour

In [107]:
import time
from os.path import exists

def run_all():
    instance_filenames = ['pr107.tsp', 'pr152.tsp', 'd198.tsp', 'pr439.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp']
    for test, filename in enumerate(instance_filenames):
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Тест {} из файла {}:'.format(test + 1, filename), end='\n')
        time_start = time.monotonic()
        quality_nn = calculate_tour_length(instance, solve_tsp_nearest_neighbour(instance))
        time_nn = time.monotonic()-time_start
        time_start = time.monotonic()
        quality_ni = calculate_tour_length(instance, solve_tsp_nearest_insertion(instance))
        time_ni = time.monotonic()-time_start
        print('Алгоритм Nearest Neighbour: длина тура {}, вычислено за {:.2} секунд'.format(int(quality_nn), time_nn), end = '\n')
        print('Алгоритм Nearest Insertion: длина тура {}, вычислено за {:.2} секунд'.format(int(quality_ni), time_ni), end = '\n\n')

In [108]:
run_all()

Тест 1 из файла pr107.tsp:
Алгоритм Nearest Neighbour: длина тура 47464, вычислено за 0.0063 секунд
Алгоритм Nearest Insertion: длина тура 52587, вычислено за 0.068 секунд

Тест 2 из файла pr152.tsp:
Алгоритм Nearest Neighbour: длина тура 85314, вычислено за 0.011 секунд
Алгоритм Nearest Insertion: длина тура 88530, вычислено за 0.073 секунд

Тест 3 из файла d198.tsp:
Алгоритм Nearest Neighbour: длина тура 18830, вычислено за 0.019 секунд
Алгоритм Nearest Insertion: длина тура 17631, вычислено за 0.16 секунд

Тест 4 из файла pr439.tsp:
Алгоритм Nearest Neighbour: длина тура 131702, вычислено за 0.091 секунд
Алгоритм Nearest Insertion: длина тура 130067, вычислено за 0.86 секунд

Тест 5 из файла d493.tsp:
Алгоритм Nearest Neighbour: длина тура 44160, вычислено за 0.13 секунд
Алгоритм Nearest Insertion: длина тура 39982, вычислено за 1.1 секунд

Тест 6 из файла d657.tsp:
Алгоритм Nearest Neighbour: длина тура 62860, вычислено за 0.22 секунд
Алгоритм Nearest Insertion: длина тура 57906, в

## Выводы
Исходя из результатов тестов, эффективность и точность предложенных алгоритмов сильно зависит от конструкции графа. В каких-то случаях лучше **Nearest Neighbour**, в каких-то **Nearest Insertion**. Зачастую **Nearest Insertion** находит тур ощутимо меньшего веса, но при этом его временная сложность составляет $O(n^3)$ против $O(n^2)$ у **Nearest Neighbour**.